In [97]:
import pandas as pd
import numpy as np
import datetime

## Loading Dataset

In [98]:
print("Loading Dataset...")

try:
    df = pd.read_csv('Online_Retail_2.csv', encoding='latin1')
    print("Dataset Loaded Successfully.")
except FileNotFoundError:
    print("Error: The file 'Online_Retail_2.csv' was not found.")
    print("please make sure that the file is in the same directory as the script.")
    exit()

Loading Dataset...
Dataset Loaded Successfully.


## Understanding the Dataset Structure

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [100]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [101]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [102]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0


In the above summary of descriptive statistics, the minimum value for `Quantity` and `UnitPrice` columns is negative which indicates that the dataset includes cancelled orders for which the `Quantity` and `UnitPrice` are negative.

## Data Cleaning and Preprocessing

### Handling Duplicate Rows

In [103]:
df.duplicated().sum()

5268

These `5268` duplicate rows need to be removed to ensure data integrity and prevent skewed analysis.

In [104]:
print("Handling duplicate rows...")
initial_row_count = len(df)
df.drop_duplicates(inplace=True)
print(f"Removed {initial_row_count - len(df)} duplicate rows from the dataset.")
print(f"Remaining row count: {len(df)}")

Handling duplicate rows...
Removed 5268 duplicate rows from the dataset.
Remaining row count: 536641


### Handling the Missing Values

In [105]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135037
Country             0
dtype: int64

The `CustomerID` column need to be removed from the dataset. As it contains `135037` missing values, which represents approximately one-quarter of the total rows. Dropping these rows would result in an incomplete and potentially inaccurate view of total sales, which could distort inventory forecasts and lead to stockouts or excess inventory.

In [106]:
df.drop(columns=['CustomerID'], inplace=True)

In [107]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,United Kingdom


For `Description`, the rows with missing values need to be removed as it's a key identifier for products.

In [108]:
df.dropna(subset=['Description'], inplace=True)

In [109]:
print("Missing values after cleaning:\n", df.isna().sum())
print(f"\nRow count after handling the missing values: {len(df)}")

Missing values after cleaning:
 InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
Country        0
dtype: int64

Row count after handling the missing values: 535187


### Converting Data Types

The `InvoiceDate` is currently an object (string). It needs to be converted to a datetime object to perform time-series analysis and feature engineering correctly.

In [110]:
print("Converting the data types...")
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
print("Converted 'InvoiceDate' column to datetime object.")
df.info()

Converting the data types...
Converted 'InvoiceDate' column to datetime object.
<class 'pandas.core.frame.DataFrame'>
Index: 535187 entries, 0 to 541908
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    535187 non-null  object        
 1   StockCode    535187 non-null  object        
 2   Description  535187 non-null  object        
 3   Quantity     535187 non-null  int64         
 4   InvoiceDate  535187 non-null  datetime64[ns]
 5   UnitPrice    535187 non-null  float64       
 6   Country      535187 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 32.7+ MB


### Handling Anomalies (Negative Quantities and Unit Prices)

Based on the descriptive statistics summary, the minimum values for the `Quantity` and `UnitPrice` columns are negative. This suggests the dataset includes cancelled or returned orders, where the quantity and unit price are represented as negative values. For a sales prediction model, we need to focus on positive sales transactions. Therefore, rows where `Quantity` is less than or equal to 0 need to be filtered out. Similarly, rows with a non-positive `UnitPrice` must also be filtered out, as a product cannot be sold for a zero or negative price.

In [111]:
print("Handling anomalies in the dataset...")
negative_quantity_count = (df['Quantity'] <= 0).sum()
df = df[df['Quantity'] > 0]
print(f"Removed {negative_quantity_count} rows with non-positive values (i.e., return orders or cancellations) in the 'Quantity' column.")
negative_unit_price_count = (df['UnitPrice'] <= 0).sum()
df = df[df['UnitPrice'] > 0]
print(f"Removed {negative_unit_price_count} rows with non-positive values (i.e., return orders and cancellatons) in the 'UnitPrice' column.")
print(f"Row count after handling anomalies in the dataset: {len(df)}")

Handling anomalies in the dataset...
Removed 9725 rows with non-positive values (i.e., return orders or cancellations) in the 'Quantity' column.
Removed 584 rows with non-positive values (i.e., return orders and cancellatons) in the 'UnitPrice' column.
Row count after handling anomalies in the dataset: 524878


## Feature Engineering

#### 1. Total Sales:
The total sales amount for each line needs to be calculated. This is a fundamental feature that gives us the monetary value of each transaction.

In [112]:
df['TotalSales'] = df['Quantity'] * df['UnitPrice']
print("Added 'TotalSales' column to the dataset.")
df.head()

Added 'TotalSales' column to the dataset.


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country,TotalSales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,United Kingdom,20.34


#### 2. Time-based Features
The information from the `InvoiceDate` column is extracted into separate features for year, month, week, and day. These features are essential for capturing seasonality and other time-related patterns within the dataset.

In [113]:
df['Invoice_Date'] = df['InvoiceDate'].dt.date
df['InvoiceYear'] = df['InvoiceDate'].dt.year
df['InvoiceMonth'] = df['InvoiceDate'].dt.month
df['InvoiceWeek'] = df['InvoiceDate'].dt.isocalendar().week.astype(int)
df['WeekStartDate'] = df.apply(
    lambda row: datetime.date.fromisocalendar(row['InvoiceYear'], row['InvoiceWeek'], 1), axis=1
)
df['IncoiceDay'] = df['InvoiceDate'].dt.day_name()
print("Extracted time-based features year, month, week, and day from 'InvoiceDate' and added them as columns to the dataset. Also, removed the time from the 'InvoiceDate' to convert in into date and added it as a 'Invoice_Date' column.")
df.head()

Extracted time-based features year, month, week, and day from 'InvoiceDate' and added them as columns to the dataset. Also, removed the time from the 'InvoiceDate' to convert in into date and added it as a 'Invoice_Date' column.


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country,TotalSales,Invoice_Date,InvoiceYear,InvoiceMonth,InvoiceWeek,WeekStartDate,IncoiceDay
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,United Kingdom,15.30,2010-12-01,2010,12,48,2010-11-29,Wednesday
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,United Kingdom,20.34,2010-12-01,2010,12,48,2010-11-29,Wednesday
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,United Kingdom,22.00,2010-12-01,2010,12,48,2010-11-29,Wednesday
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,United Kingdom,20.34,2010-12-01,2010,12,48,2010-11-29,Wednesday
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,United Kingdom,20.34,2010-12-01,2010,12,48,2010-11-29,Wednesday


## Creating Weekly Shipment Data

To transform the transaction data into a clean, weekly time-series format for each product, we will first create a DataFrame with the necessary columns: `StockCode`, `InvoiceDate`, and `Quantity`. After setting `InvoiceDate` as the index, we will loop through each unique product. For each product, we will resample the data to a weekly frequency, sum the quantities, and then join it to a complete date range to fill in missing weeks. The missing values will then be filled with zeros to ensure a consistent history. Finally, we will add the product code, rename the quantity column, and append it to our main DataFrame for a clean output.

In [114]:
df_weekly = df[['StockCode', 'WeekStartDate', 'Quantity']].copy()
all_products = df_weekly['StockCode'].unique()
min_date = df_weekly['WeekStartDate'].min()
max_date = df_weekly['WeekStartDate'].max()
date_range = pd.date_range(start=min_date, end=max_date, freq='W-MON')  # Weekly on Monday

clean_weekly_data = pd.DataFrame()
for product_code in all_products:
    product_data = df_weekly[df_weekly['StockCode'] == product_code].copy()
    product_weekly_sum = product_data.groupby('WeekStartDate')['Quantity'].sum()
    product_resampled = product_weekly_sum.reindex(date_range, fill_value=0).to_frame(name='WeeklyShipments')
    product_resampled['StockCode'] = product_code
    clean_weekly_data = pd.concat([clean_weekly_data, product_resampled])

clean_weekly_data.reset_index(inplace=True)
clean_weekly_data.rename(columns={'index': 'WeekStartDate'}, inplace=True)
clean_weekly_data.head()

,WeekStartDate,WeeklyShipments,StockCode
0,2010-11-29,986,85123A
1,2010-12-06,1045,85123A
2,2010-12-13,1524,85123A
3,2010-12-20,197,85123A
4,2010-12-27,0,85123A


In [115]:
clean_weekly_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211788 entries, 0 to 211787
Data columns (total 3 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   WeekStartDate    211788 non-null  datetime64[ns]
 1   WeeklyShipments  211788 non-null  int64         
 2   StockCode        211788 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 4.8+ MB


## Saving the Final Dataset as a New Excel File

In [116]:
print("Saving the final dataset as a new Excel file...")
output_file_name = 'Proccessed_Online_Retail_2.xlsx'
try:
    with pd.ExcelWriter(output_file_name, engine = 'openpyxl') as writer:
        df.to_excel(writer, sheet_name = 'Cleaned_processed_dataset', index = False)
        clean_weekly_data.to_excel(writer, sheet_name = 'Weekly_Shipment_Data', index = False)
    print(f"Dataset saved successfully to '{output_file_name}' in two sheets.")
except Exception as e:
    print(f"error: An error occurred while saving the file. {e}")
print("Data Cleaning and Preprocessing Completed. The dataset is ready for Exploratory Data Analysis and Sales Forecasting.")

Saving the final dataset as a new Excel file...
Dataset saved successfully to 'Proccessed_Online_Retail_2.xlsx' in two sheets.
Data Cleaning and Preprocessing Completed. The dataset is ready for Exploratory Data Analysis and Sales Forecasting.
